In [1]:
import ndcctools.taskvine as vine
import json
import os

In [2]:
!ls data/parquet/10split

group_1.parquet   group_3.parquet  group_6.parquet  group_9.parquet
group_10.parquet  group_4.parquet  group_7.parquet
group_2.parquet   group_5.parquet  group_8.parquet


In [3]:
import subprocess

def get_local_ip():
    try:
        # Run the 'hostname -I' command
        result = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # Get the output, strip any extra spaces or newlines
        ip_addresses = result.stdout.strip()
        return ip_addresses
    except subprocess.CalledProcessError as e:
        return f"Error: {e}"

# Get and print the local IP addresses
local_ips = get_local_ip()
print(f"My local IP addresses are: {local_ips}")
print(local_ips.split()[0])
managerIp = local_ips.split()[0]

My local IP addresses are: 10.140.86.137 169.254.95.120 10.0.86.137
10.140.86.137


In [4]:
slurm_script = """#!/bin/bash
#SBATCH --job-name=taskvine           
#SBATCH --output=result.out           
#SBATCH --account=nirav
#SBATCH --partition=standard           
#SBATCH --nodes=1
#SBATCH --ntasks=2                    
#SBATCH --time=01:00:00               

# Load any necessary modules
module load python/3.9
source ~/.bashrc
conda init bash
conda init
conda activate taskvine

# Execute the Python script or command
vine_submit_workers -T slurm -p "--cpus-per-task=4 --time=1:00:00" -t 01:00:00 """+managerIp+""" 9124 10

"""

In [5]:
slurm_filename = "taskvine.slurm"
with open(slurm_filename, "w") as slurm_file:
    slurm_file.write(slurm_script)

print(f"SLURM script saved as {slurm_filename}")

SLURM script saved as taskvine.slurm


In [6]:
%%bash
python bin/TaskvineLDAWT.py &
sbatch taskvine.slurm

Submitted batch job 2146022
Listening on port 9124
Waiting for tasks to complete...
100%|██████████| 10000/10000 [01:04<00:00, 154.96it/s]
Completed with 78 errors.
Tared output folder into: /tmp/worker-2299-11626/task.8/group_8.tar.gz

100%|██████████| 10000/10000 [00:47<00:00, 211.49it/s]
Completed with 55 errors.
Tared output folder into: /tmp/worker-2299-19734/task.9/group_6.tar.gz

100%|██████████| 10000/10000 [01:43<00:00, 97.00it/s]
Completed with 83 errors.
Tared output folder into: /tmp/worker-2299-9400/task.10/group_9.tar.gz

100%|██████████| 10000/10000 [00:42<00:00, 235.81it/s]
Completed with 58 errors.
Tared output folder into: /tmp/worker-2299-1339/task.3/group_4.tar.gz

100%|██████████| 10000/10000 [01:10<00:00, 140.94it/s]
Completed with 49 errors.
Tared output folder into: /tmp/worker-2299-11624/task.4/group_7.tar.gz

100%|██████████| 10000/10000 [01:57<00:00, 84.96it/s]
Completed with 44 errors.
Tared output folder into: /tmp/worker-2299-9401/task.2/group_5.tar.gz

10